In [1]:
from bloqade import squin
from kirin.dialects.ilist import IList
import numpy as np
import bloqade.stim
import bloqade.tsim
from manual_noise_injection import configure_poi, get_poi_kernel
import numpy as np


In [2]:
def MSD_encoding(theta, phi,basis="z"):
    # @squin.kernel
    # def circ():
    #     q=squin.qalloc(14)
    #     parameterized_MSD_encoding([q[i] for i in range(7)],theta=theta,phi=phi)
    #     parameterized_MSD_encoding([q[i+7] for i in range(7)],theta=np.pi/2,phi=0)

    @squin.kernel
    def parameterized_MSD_encoding():
        q=squin.qalloc(7)
        squin.u3(theta,phi,0,q[6])
        for i in range(6):
            squin.sqrt_y_adj(q[i])
        # [squin.broadcast.sqrt_y_adj(q[i]) for i in range(5)]
        squin.cz(q[1], q[2])
        squin.cz(q[3], q[4])
        squin.cz(q[5], q[6])
        squin.sqrt_y(q[6])
        squin.cz(q[0],q[3])
        squin.cz(q[2],q[5])
        squin.cz(q[4],q[6])
        for i in range(5):
            squin.sqrt_y(q[i+2])
        # [squin.broadcast.sqrt_y(q[i+2]) for i in range(5)]
        squin.cz(q[0],q[1])
        squin.cz(q[2],q[3])
        squin.cz(q[4],q[5])
        squin.sqrt_y(q[1])
        squin.sqrt_y(q[2])
        squin.sqrt_y(q[4])
        squin.broadcast.measure(q)
    
    return parameterized_MSD_encoding

In [3]:
MSD_enc = MSD_encoding(0,0)
tsim_circ = bloqade.tsim.Circuit(MSD_enc)
tsim_circ.diagram(height=400)

In [ ]:
# Step 1: Configure your POIs at the top
# poi = configure_poi({
#     "a": ("depolarize", 0.5),
#     "interesting_area": ("qubit_loss", 0.5),
#     "after_cz": ("depolarize", 0.1),
# }) 


def MSD_encoding_plus_ancilla(theta, phi,lam,basis="z"):

    # poi_a = get_poi_kernel("a")
    # poi_interesting = get_poi_kernel("interesting_area")
    # poi_after_cz = get_poi_kernel("after_cz")

    @squin.kernel
    def parameterized_MSD_encoding(q_subset,theta,phi,lam):
        squin.u3(theta,phi,lam,q_subset[6])
        for i in range(6):
            squin.sqrt_y_adj(q_subset[i])
        squin.cz(q_subset[1], q_subset[2])
        squin.cz(q_subset[3], q_subset[4])
        squin.cz(q_subset[5], q_subset[6])

        # poi_a(q_subset[1])

        squin.sqrt_y(q_subset[6])
        squin.cz(q_subset[0],q_subset[3])
        squin.cz(q_subset[2],q_subset[5])
        squin.cz(q_subset[4],q_subset[6])
        for i in range(5):
            squin.sqrt_y(q_subset[i+2])
        # [squin.broadcast.sqrt_y(q[i+2]) for i in range(5)]
        squin.cz(q_subset[0],q_subset[1])
        squin.cz(q_subset[2],q_subset[3])
        squin.cz(q_subset[4],q_subset[5])
        squin.sqrt_y(q_subset[1])
        squin.sqrt_y(q_subset[2])
        squin.sqrt_y(q_subset[4])
        
    @squin.kernel
    def circ():
        q=squin.qalloc(14)
        #blah([q[0],q[1]])
        parameterized_MSD_encoding([q[0],q[1],q[2],q[3],q[4],q[5],q[6]],theta=theta,phi=phi,lam=lam)
        parameterized_MSD_encoding([q[7],q[8],q[9],q[10],q[11],q[12],q[13]],theta=np.pi/2,phi=0,lam=np.pi)
        
    
    return circ

In [33]:
MSD_enc = MSD_encoding_plus_ancilla(1,0,0)
tsim_circ = bloqade.tsim.Circuit(MSD_enc)
tsim_circ.diagram(height=400)